In [87]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [88]:
orignal_data = pandas.read_csv('problem_2_train.csv' , index_col=0)
testdata = pandas.read_csv('problem_2_test.csv', index_col=0)

In [89]:
def preprocess(data):
    label_encoder = preprocessing.LabelEncoder() 
    data['mobmake']= label_encoder.fit_transform(data['mobmake']) 
    data['custpincodecategory']= label_encoder.fit_transform(data['custpincodecategory']) 
    data['claimtype']= label_encoder.fit_transform(data['claimtype']) 
    data['memcreateddate']=pandas.to_datetime(data['memcreateddate'], errors='coerce', format='%Y-%m-%d')
    data['mobilepurchasedate']=pandas.to_datetime(data['mobilepurchasedate'], errors='coerce', format='%Y-%m-%d')
    data['claimintdate']=pandas.to_datetime(data['claimintdate'], errors='coerce', format='%Y-%m-%d')
    data['pickupStartdate']=pandas.to_datetime(data['pickupStartdate'], errors='coerce', format='%Y-%m-%d')
    data['memandpurchasediff']=((data['memcreateddate']-data['mobilepurchasedate']).dt.days).astype(int)
    data['claimpurchasediff']=((data['claimintdate']-data['mobilepurchasedate']).dt.days).astype(int)
    data['pickupclaimdiff']=((data['pickupStartdate']-data['claimintdate']).dt.days).astype(int)
    data['pickanddeliversame']=np.where(data['pickuppartnerid']==data['deliverypartnerid'] ,1 ,0 )
    data['pincode']=np.where(data['custpincode']==data['hubpincode'],0,data['hubpincode'])
    data.drop(['memcreateddate' ,'mobilepurchasedate' , 'claimintdate' ,'pickupStartdate'] ,axis=1 , inplace = True) 
    names = data.columns
    scaler = StandardScaler()
    text=scaler.fit_transform(data)
    data_scaled =pandas.DataFrame(text, columns=names)
    #data_scaled.head()
    return data_scaled

In [127]:
def featureextraction(d):
    feature_cols = ['mobmake', 'hubid',
       'pickanddeliversame', 'insuranceCompanyId', 'custpincodecategory',
       'claimtype',  'servicecntrid' , "memandpurchasediff","claimpurchasediff","pickupclaimdiff" ]
    X=d[feature_cols]
    return X

In [90]:
data=preprocess(orignal_data)
testing=preprocess(testdata)
X=featureextraction(data)
y_orignal=orignal_data['tat_in_days']
y=data['tat_in_days']

In [126]:
mean_of_array = orignal_data.mean(axis=0)
std_of_array = orignal_data.std(axis=0)

In [138]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train ,X_test , y_train , y_test = train_test_split(X , y , random_state=1) 
line  = LinearRegression(normalize=True)
line.fit(X_train, y_train)
y_pred = line.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test , y_pred)))

0.994769608668839


In [129]:
test_features= featureextraction(testing)
test_predict = line.predict(test_features)

In [130]:
ans=(test_predict*std_of_array['tat_in_days'])+mean_of_array['tat_in_days']

In [131]:
pandas.DataFrame(ans ,index=None, columns=['Prediction'] ).to_csv('results.csv')